In [61]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Importando os dataframes de predições

In [62]:
file_path = '/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_experimento/e1.1/treino_com_desistentes.xlsx'
df_rf = pd.read_excel(file_path, sheet_name='random_forest')
df_etc = pd.read_excel(file_path, sheet_name='extra_tree_classifier')
df_xgb_we = pd.read_excel(file_path, sheet_name='xgboost_com_early_stopping')
df_xgb_no = pd.read_excel(file_path, sheet_name='xgboost_sem_early_stopping')
df_gnb = pd.read_excel(file_path, sheet_name='xgboost_sem_early_stopping')


file_path = '/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_treino/turma_102_2016_1.1/treino.xlsx'
df1 = pd.read_excel(file_path, sheet_name='exam_0')
df2 = pd.read_excel(file_path, sheet_name='exam_1')
df3 = pd.read_excel(file_path, sheet_name='exam_2')
df4 = pd.read_excel(file_path, sheet_name='exam_3')
df5 = pd.read_excel(file_path, sheet_name='exam_4')
df6 = pd.read_excel(file_path, sheet_name='exam_5')


## Pré-processando os dados

In [63]:
df_rf.drop(['Unnamed: 0'], inplace=True, axis=1)
df_etc.drop(['Unnamed: 0'], inplace=True, axis=1)
df_xgb_we.drop(['Unnamed: 0'], inplace=True, axis=1)
df_xgb_no.drop(['Unnamed: 0'], inplace=True, axis=1)
df_gnb.drop(['Unnamed: 0'], inplace=True, axis=1)

df1.fillna(0, inplace=True)
df2.fillna(0, inplace=True)
df3.fillna(0, inplace=True)
df4.fillna(0, inplace=True)
df5.fillna(0, inplace=True)
df6.fillna(0, inplace=True)

def activity(row):
    if np.sum(row) > 0:
        return 'Ativo'
    else:
        return 'Inativo'

df1['status_1'] = df1.iloc[:, 3:].apply(lambda row: activity(row), axis='columns')
df2['status_2'] = df2.iloc[:, 3:].apply(lambda row: activity(row), axis='columns')
df3['status_3'] = df3.iloc[:, 3:].apply(lambda row: activity(row), axis='columns')
df4['status_4'] = df4.iloc[:, 3:].apply(lambda row: activity(row), axis='columns')
df5['status_5'] = df5.iloc[:, 3:].apply(lambda row: activity(row), axis='columns')
df6['status_6'] = df6.iloc[:, 3:].apply(lambda row: activity(row), axis='columns')

## Juntando todos os status

In [70]:
df_status = pd.concat([df1[['status_1']], df2[['status_2']], df3[['status_3']], df4[['status_4']], df5[['status_5']], df6[['status_6']]], axis=1)

## Cruzando as informações

In [44]:
df_rf.loc[(df_rf['target'] == 'Desistente') &
          (df_rf['exam_1'] == 'Desistente')]

,target,exam_1,exam_2,exam_3,exam_4,exam_5,exam_6
0,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente
2,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente
4,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente
18,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente
24,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente
28,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente
34,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente
39,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente,Desistente


In [132]:
def metrics(df_predic, df_status):
    l = [['D_D', 'DN_D', 'D_ND', 'ND_ND', 'ASD_predicto', 'ASND_predicto', 'ASD_real', 'ASND_real']]
    for e, s in zip(df_predic.iloc[:,1:].columns, df_status.columns):
        m = [[]]
        tp = df_predic.loc[(df_predic['target'] == 'Desistente') & (df_predic[e] == 'Desistente')].shape[0]
        tn = df_predic.loc[(df_predic['target'] == 'Não Desistente') & (df_predic[e] == 'Não Desistente')].shape[0]
        fp = df_predic.loc[(df_predic['target'] == 'Não Desistente') & (df_predic[e] == 'Desistente')].shape[0]
        fn = df_predic.loc[(df_predic['target'] == 'Desistente') & (df_predic[e] == 'Não Desistente')].shape[0]
        
        i   = df_predic.loc[df_predic[e] == 'Desistente'][e].index
        df  = df_status.loc[i, s]
        asd_p = df[df == 'Inativo'].shape[0]
        i   = df_predic.loc[df_predic[e] == 'Não Desistente'][e].index
        df  = df_status.loc[i, s]
        asnd_p = df[df == 'Inativo'].shape[0]
        i   = df_predic.loc[df_predic['target'] == 'Desistente'][e].index
        df  = df_status.loc[i, s]
        asd_r = df[df == 'Inativo'].shape[0]
        i   = df_predic.loc[df_predic['target'] == 'Não Desistente'][e].index
        df  = df_status.loc[i, s]
        asnd_r = df[df == 'Inativo'].shape[0]
        
        m = [tp, fn, fp, tn, asd_p, asnd_p, asd_r, asnd_r]
        l.append(m)

    ind = ['exam_'+str(x) for x in range(1,7)]
    df = pd.DataFrame(l[1:], columns=l[0], index=ind)
    return df

In [133]:
df_metrics_rf = metrics(df_rf, df_status)
df_metrics_etc = metrics(df_etc, df_status)
df_metrics_xgb_we = metrics(df_xgb_we, df_status)
df_metrics_xgb_no = metrics(df_xgb_no, df_status)
df_metrics_gnb = metrics(df_gnb, df_status)

In [134]:
from pandas import ExcelWriter

file_path = '/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_experimento/e1.1/metrics.xlsx'

writer = ExcelWriter(file_path)

df_metrics_rf.to_excel(writer, sheet_name='random_forest')
df_metrics_etc.to_excel(writer, sheet_name='extra_tree_classifier')
df_metrics_xgb_we.to_excel(writer, sheet_name='xgboost_com_early_stopping')
df_metrics_xgb_no.to_excel(writer, sheet_name='xgboost_sem_early_stopping')
df_metrics_gnb.to_excel(writer, sheet_name='gaussian_naive_bayes')

writer.save()